In [1]:
import spacy, stanza
import pandas as pd
from collections import defaultdict


/Users/perrine/Desktop/Master_S2/OutilTraitementCorpus/otc_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading SpaCy models
nlp_en = spacy.load("en_core_web_sm")  # English
nlp_es = spacy.load("es_core_news_sm")  # Spanish
nlp_de = spacy.load("de_core_news_sm")  # German
nlp_fr = spacy.load("fr_core_news_sm")  # French
nlp_ru = spacy.load("ru_core_news_sm")  # Russian
nlp_zh = spacy.load("zh_core_web_sm")  # Chinese
nlp_ja = spacy.load("ja_core_news_sm")  # Japanese
nlp_ko = spacy.load("ko_core_news_sm")  # Korean

nlp_ar = stanza.Pipeline(lang='ar')

2024-04-23 21:24:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-23 21:24:35 INFO: Downloaded file to /Users/perrine/stanza_resources/resources.json
2024-04-23 21:24:36 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-04-23 21:24:36 INFO: Using device: cpu
2024-04-23 21:24:36 INFO: Loading: tokenize
2024-04-23 21:24:36 INFO: Loading: mwt
2024-04-23 21:24:36 INFO: Loading: pos
2024-04-23 21:24:36 INFO: Loading: lemma
2024-04-23 21:24:36 INFO: Loading: depparse
2024-04-23 21:24:36 INFO: Loading: ner
2024-04-23 21:24:37 INFO: Done loading processors!


In [6]:
def tokenize(text, lang):
    if lang == 'ar':
        doc = nlp_ar(text)
        return [word.text for sent in doc.sentences for word in sent.words]
    else:
        nlp = globals()[f"nlp_{lang}"]
        doc = nlp(text)
        return [token.text for token in doc]

**Tokenisation** : 
- Chaque document est tokenisé selon la langue à l'aide des modèles de NLP correspondants. Les tokens sont les mots dans le cas des langues occidentales et des unités plus complexes pour des langues comme le chinois ou le japonais.

In [7]:
def calculate_metrics(data):
    language_data = defaultdict(list)

    for index, row in data.iterrows():
        tokens = tokenize(row['Text'], row['Label'])
        language_data[row['Label']].append(tokens)
    
    results = {}

    for lang, tokens_list in language_data.items():
        total_words = sum(len(tokens) for tokens in tokens_list)
        total_samples = len(tokens_list)
        unique_words = len(set(word for tokens in tokens_list for word in tokens))
        
        average_words_per_sample = total_words / total_samples if total_samples else 0
        lexical_diversity = unique_words / total_words if total_words else 0

        results[lang] = {
            'Average Words per Sample': average_words_per_sample,
            'Lexical Diversity': lexical_diversity
        }

    return results

**Métriques calculées** :
- **Moyenne des mots par échantillon** : Cela donne une idée de la longueur moyenne des textes pour chaque langue.
- **Diversité lexicale** : C'est le rapport entre le nombre de mots uniques et le nombre total de mots dans les échantillons de chaque langue. Une valeur plus élevée indique une plus grande richesse lexicale.

In [8]:
# Load data
data = pd.read_csv('../../data/csv/data_balanced.csv')

# Compute metrics
metrics = calculate_metrics(data)

# Display the metrics
print(metrics)

{'ko': {'Average Words per Sample': 191.4, 'Lexical Diversity': 0.625914315569488}, 'ar': {'Average Words per Sample': 102.2, 'Lexical Diversity': 0.46868884540117417}, 'fr': {'Average Words per Sample': 1230.9, 'Lexical Diversity': 0.2210577626127224}, 'zh': {'Average Words per Sample': 501.7, 'Lexical Diversity': 0.3060593980466414}, 'es': {'Average Words per Sample': 389.95, 'Lexical Diversity': 0.3341454032568278}, 'ru': {'Average Words per Sample': 1028.65, 'Lexical Diversity': 0.3505079473095805}, 'de': {'Average Words per Sample': 502.55, 'Lexical Diversity': 0.3381753059397075}, 'ja': {'Average Words per Sample': 593.2, 'Lexical Diversity': 0.2226062036412677}, 'en': {'Average Words per Sample': 1418.55, 'Lexical Diversity': 0.20584399562933983}}


**Affichage des Résultats** : 
- Les résultats sont stockés dans un dictionnaire et imprimés. Vous pouvez également les visualiser en utilisant matplotlib ou une autre bibliothèque de visualisation pour mieux comprendre les distributions.

En analysant les données fournies sur la reconnaissance de la langue d'un texte à partir des pages Wikipédia, je m'intéresse particulièrement au nombre moyen de mots par échantillon et à la diversité lexicale de chaque langue. Ces indicateurs me permettent d'évaluer la complexité et la richesse du vocabulaire utilisé dans ces corpus linguistiques divers. Voici mon interprétation personnelle des résultats obtenus pour chaque langue :

1. **Coréen (ko)** : Le coréen affiche un nombre moyen de mots par échantillon de 191.4 et une diversité lexicale de 0.626. Ce niveau élevé de diversité lexicale suggère une grande variété de mots utilisés, même dans des textes plus courts, ce qui reflète la complexité structurelle de la langue coréenne.

2. **Arabe (ar)** : Avec une moyenne de 102.2 mots par échantillon et une diversité lexicale de 0.469, l'arabe semble utiliser un vocabulaire varié, même dans un contexte de rédaction relativement concis. Cela pourrait être dû à la riche morphologie de l'arabe qui permet une grande variabilité dans l'utilisation des mots.

3. **Français (fr)** : Les textes en français sont particulièrement longs, avec une moyenne de 1230.9 mots par échantillon, mais présentent une faible diversité lexicale de 0.221. Cela indique une utilisation répétitive du vocabulaire dans de longs articles, typique des discussions approfondies sur des sujets complexes.

4. **Chinois (zh)** : Le chinois montre une moyenne de 501.7 mots par échantillon avec une diversité lexicale de 0.306. Je trouve que cette langue utilise efficacement un vocabulaire varié même dans des textes de longueur modérée, ce qui est caractéristique de ses traits logographiques.

5. **Espagnol (es)** : L'espagnol présente une moyenne de mots par échantillon de 389.95 avec une diversité lexicale de 0.334. J'observe un bon équilibre entre la longueur des textes et une variété raisonnable de vocabulaire, ce qui pourrait refléter une tendance à la descriptivité tout en restant accessible.

6. **Russe (ru)** : Avec une moyenne élevée de 1028.65 mots et une diversité lexicale de 0.351, le russe semble exploiter un vocabulaire riche dans des descriptions détaillées, ce que j'attribue à la nature descriptive de la langue et à son système morphologique complexe.

7. **Allemand (de)** : Les textes allemands, avec 502.55 mots en moyenne et une diversité lexicale de 0.338, montrent une préférence pour l'utilisation de vocabulaire varié dans des textes de longueur moyenne. Je pense que cela est dû à la capacité de l'allemand à former des mots composés, enrichissant ainsi son lexique.

8. **Japonais (ja)** : Avec 593.2 mots par échantillon et une diversité lexicale de 0.223, le japonais semble favoriser des textes modérément longs avec une utilisation répétitive de termes spécifiques. Cette caractéristique reflète, à mon avis, la structure grammaticale et le style formel du japonais.

9. **Anglais (en)** : L'anglais se distingue avec le plus haut nombre moyen de mots (1418.55) et la diversité lexicale la plus basse (0.206). Cela me suggère une tendance à des textes très longs avec une utilisation limitée de vocabulaire diversifié, ce qui peut indiquer une norme de rédaction détaillée et exhaustive typique de Wikipédia en anglais.

En résumé, ces observations montrent comment chaque langue utilise ses structures et son vocabulaire pour refléter ses caractéristiques uniques. Les différences notées dans la longueur des textes et la diversité lexicale entre les langues peuvent également être influencées par les spécificités des articles de Wikipédia, qui varient selon le sujet et le style de rédaction propre à chaque communauté linguistique.